In [1]:
from hpmetad import HPMetaD
import hoomd
import gsd.hoomd
import numpy as np
from hoomd import hpmc
import time
import coxeter

In [2]:
# Define shape
alpha_a = 0.1
alpha_c = 0.3

a = alpha_a * (3 - 1) + 1
c = alpha_c * (3 - 1) + 1
target_pf = 0.6

name = f'shape_a_{int(alpha_a * 100)}_c_{int(alpha_c * 100)}'
shape = coxeter.families.Family323Plus.get_shape(a=a, c=c)
shape.volume = 1
shape.center = [0, 0, 0]

verts = shape.vertices.tolist()

In [3]:
# Simulation parameters
run_params = dict(N_random = 1e3, N_compress = 1e3, N_tune = 2e3, N_eq = 1e6)
fname = f'{name}_pf_{target_pf}'
comm = hoomd.communicator.Communicator()
cpu = hoomd.device.CPU(communicator=comm, notice_level=2)

In [4]:
seed = int(np.genfromtxt(f'./data/output_{fname}.txt', usecols=2)[-1])
sim = hoomd.Simulation(device=cpu, seed=seed)
sim.create_state_from_gsd(filename=f'./data/restart_{fname}.gsd')
print(f'Current timestep: {sim.timestep}')

Current timestep: 1004000


In [5]:
mc_a = np.genfromtxt(f'./data/output_{fname}.txt', usecols=6)[-1]
mc_d = np.genfromtxt(f'./data/output_{fname}.txt', usecols=7)[-1]
print(f'MC a move size:{mc_a}')
print(f'MC d move size:{mc_d}')

MC a move size:0.06805
MC d move size:0.05185


In [6]:
mc = hpmc.integrate.ConvexPolyhedron()
mc.shape[name] = dict(vertices=verts)

p_types = sim.state.particle_types
for p_type in p_types:
    mc.a[p_type] = mc_a
    mc.d[p_type] = mc_d

sim.operations.integrator = mc

In [7]:
dump_period = 1e2
log_period = 5e2

table_logger = hoomd.logging.Logger(categories=['scalar', 'string'])
table_logger.add(sim, quantities=['timestep', 'tps', 'seed', 'walltime'])

def add_quantities_to_logger(custom_logger):
    custom_logger[('overlaps', )] = (lambda: mc.overlaps, 'scalar')
    custom_logger[('box_volume', )] = (lambda: sim.state.box.volume, 'scalar')

    # Log MC params
    for p_type in p_types:
        custom_logger[(f'mc_a_{p_type}',)] = (lambda: mc.a[p_type], 'scalar')
        custom_logger[(f'mc_d_{p_type}',)] = (lambda: mc.d[p_type], 'scalar')

    custom_logger[('translate_move_acceptance_ratio', )] = (
        lambda: mc.translate_moves[0] / sum(mc.translate_moves), 'scalar')
    custom_logger[('rotate_move_acceptance_ratio', )] = (
        lambda: mc.rotate_moves[0] / sum(mc.rotate_moves), 'scalar')

add_quantities_to_logger(table_logger)
output_file = open(f'./data/output_{fname}.txt', mode='a', newline='\n')
table = hoomd.write.Table(output=output_file,
                          trigger=hoomd.trigger.Periodic(period=int(log_period)),
                          logger=table_logger)
sim.operations.writers.append(table)

logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])

In [8]:
# Restart the HPMetaD simulation
# The following parameters need to be consistent

cv_min = 0.0
cv_max = 0.5
bins = 30
init_height = 0.3
sigma = 0.002
gamma = 10

metad_stride = 10
colvar_mode = 'steinhardt'
colvar_params = dict(neighbor={'r_max': 2.0}, l=6, average=False, wl=False, weighted=False, wl_normalize=False)

In [9]:
hpmetad_action = HPMetaD(
    sim=sim,
    colvar_mode=colvar_mode,
    colvar_params = colvar_params,
    kT=1,
    init_height=init_height,
    metad_stride=metad_stride,
    sigma=sigma,
    gamma=gamma,
    cv_min=cv_min,
    cv_max=cv_max,
    bins=bins,
    seed=seed,
    restart=True,
    restart_fn='./data/WTMetaD_history.txt',
)

op = hoomd.update.CustomUpdater(action=hpmetad_action, trigger=1)
sim.operations += op
sim.operations += hpmetad_action.get_writer('./data/WTMetaD_log.hdf5')

traj_writer = hoomd.write.GSD(filename=f'./data/traj_{fname}.gsd',
                              trigger=hoomd.trigger.Periodic(int(dump_period), phase=1),
                              mode='ab',
                              dynamic=['property'],
                              logger=logger)
sim.operations.writers.append(traj_writer)
total_timestep = run_params['N_eq'] + run_params['N_random'] + run_params['N_compress'] + run_params['N_tune']

total_timestep = total_timestep + 1.6e4

while sim.timestep < total_timestep:
    step_size = int(total_timestep - sim.timestep)
    sim.run(min(1e2, step_size))
    hpmetad_action.save_history('./data/WTMetaD_history.txt')
    for writer in sim.operations.writers:
        if hasattr(writer, "flush"):
            writer.flush()

hpmetad_action.save_configuration(config_fn=f'./data/restart_{fname}.gsd', logger=logger)